In [1]:
import open3d as o3d
import numpy as np
import os
import glob
import pandas as pd


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Start building a simple tool that checks all shapes in the database. The tool should output, for each shape

-   the class of the shape
-   the number of faces and vertices of the shape
-   the type of faces (e.g. only triangles, only quads, mixes of triangles and quads)
-   the axis-aligned 3D bounding box of the shapes

In [20]:
dbpath = r"./database/"
mesh_path = dbpath + r"Sign/m1675.obj"

mesh_data = []

for class_folder in os.listdir(dbpath):
    # Construct the full path to the subfolder
    class_folder_path = os.path.join(dbpath, class_folder)
    
    # Check if the item in the directory is a directory (to skip files)
    if os.path.isdir(class_folder_path):
        # Assuming the class folder name is the class name
        class_name = class_folder
        
        # Iterate through .obj files in the subfolder
        for obj_file_path in glob.glob(os.path.join(class_folder_path, '*.obj')):
            # Read the .obj file using tripy
            obj = obj_file_path.split("\\")[-1]
            mesh = o3d.io.read_triangle_mesh(obj_file_path)
            vertices = np.asarray(mesh.vertices)
            triangles = np.asarray(mesh.triangles)
            mesh_data.append([class_name, obj, len(vertices), len(triangles), mesh])

            # print(f"Class: {class_name}, Vertices: {len(vertices)}, Triangles: {len(triangles)}")

In [21]:
df = pd.DataFrame(mesh_data, columns=["Class", "Object", "Vertices", "Triangles", "data"])
print(df)

                Class      Object  Vertices  Triangles  \
0     AircraftBuoyant   m1337.obj       201        340   
1     AircraftBuoyant   m1338.obj      8761      16960   
2     AircraftBuoyant   m1339.obj      1878       6173   
3     AircraftBuoyant   m1340.obj       778       1457   
4     AircraftBuoyant   m1341.obj       425        824   
...               ...         ...       ...        ...   
2478       WheelChair  D00687.obj      8188      16268   
2479       WheelChair  D00720.obj     43912      83378   
2480       WheelChair  D00782.obj      1134       1940   
2481       WheelChair  D00821.obj      8188      16268   
2482       WheelChair  D00846.obj      8834      17196   

                                                   data  
0       TriangleMesh with 201 points and 340 triangles.  
1     TriangleMesh with 8761 points and 16960 triang...  
2     TriangleMesh with 1878 points and 6173 triangles.  
3      TriangleMesh with 778 points and 1457 triangles.  
4       Trian

In [10]:
dfmesh = df.iloc[0]["data"]

In [11]:
o3d.visualization.draw_geometries([dfmesh])

In [12]:
df["Vertices"].min(), df["Vertices"].max(), df["Vertices"].mean()

(16, 73440, 5025.5453080950465)

In [13]:
df["Triangles"].min(), df["Triangles"].max(), df["Triangles"].mean()

(16, 129881, 10691.52597664116)

### The type of faces (e.g. only triangles, only quads, mixes of triangles and quads)

In [14]:
dbpath = r"./database/"
mesh_path = dbpath + r"Sign/m1675.obj"

mesh_data = []

for class_folder in os.listdir(dbpath):
    # Construct the full path to the subfolder
    class_folder_path = os.path.join(dbpath, class_folder)
    
    # Check if the item in the directory is a directory (to skip files)
    if os.path.isdir(class_folder_path):
        # Assuming the class folder name is the class name
        class_name = class_folder
        
        # Iterate through .obj files in the subfolder
        for obj_file_path in glob.glob(os.path.join(class_folder_path, '*.obj')):
            # Read the .obj file using tripy
            # mesh = o3d.io.read_triangle_mesh(obj_file_path)
            with open(obj_file_path, 'r') as file:
            # Iterate through each line in the file
                for line in file:
                    # Check if the line starts with 'f' (case-sensitive)
                    if line.startswith('f'):
                        if len(line.split(" "))!= 4:
                            print(len(line.split(" ")))

            # print(f"Class: {class_name}, Vertices: {len(vertices)}, Triangles: {len(triangles)}")

Only shapes of 3, otherwise it would have printed out something

### Axis-aligned bouding box

In [ ]:
mesh = o3d.io.read_triangle_mesh(mesh_path)
mesh.compute_vertex_normals()
aabb = mesh.get_axis_aligned_bounding_box()


# Get the minimum and maximum coordinates of the AABB
min_coords = aabb.get_min_bound()
max_coords = aabb.get_max_bound()

# Define the eight corners of the AABB
vertices = [
    min_coords,
    [max_coords[0], min_coords[1], min_coords[2]],
    [max_coords[0], max_coords[1], min_coords[2]],
    [min_coords[0], max_coords[1], min_coords[2]],
    [min_coords[0], min_coords[1], max_coords[2]],
    [max_coords[0], min_coords[1], max_coords[2]],
    max_coords,
    [min_coords[0], max_coords[1], max_coords[2]]
]

# Define the edges of the AABB
lines = [
    [0, 1],
    [1, 2],
    [2, 3],
    [3, 0],
    [4, 5],
    [5, 6],
    [6, 7],
    [7, 4],
    [0, 4],
    [1, 5],
    [2, 6],
    [3, 7]
]

# Create a LineSet to visualize the AABB
aabb_line_set = o3d.geometry.LineSet()
aabb_line_set.points = o3d.utility.Vector3dVector(vertices)
aabb_line_set.lines = o3d.utility.Vector2iVector(lines)

# Customize the bounding box appearance
line_set_color = np.array([1.0, 0.0, 0.0])  # Red color (R, G, B)
line_set_line_width = 5.0  # Adjust line width as needed

# Set the line set's color and line width
aabb_line_set.paint_uniform_color(line_set_color)
# aabb_line_set.line_width = line_set_line_width

# Visualize the object and its AABB
o3d.visualization.draw_geometries([mesh, aabb_line_set])